# Super Resolution Demo for Geospatial Imagery

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/super_resolution_demo.ipynb)

This notebook provides a comprehensive demonstration of the super resolution functionality in the `geoai` library. Super resolution is a technique that uses deep learning to enhance the resolution of low-resolution images, creating high-resolution outputs from low-resolution inputs.

## What is Super Resolution?

Super resolution (SR) is an image processing technique that reconstructs high-resolution images from low-resolution inputs using machine learning algorithms. In geospatial AI, this is particularly valuable for:

- **Enhancing satellite imagery**: Improving resolution of older or lower-quality satellite data
- **Temporal analysis**: Making historical imagery comparable to modern high-resolution data
- **Resource efficiency**: Reducing the need for expensive high-resolution satellite acquisitions
- **Environmental monitoring**: Better detection of small features in land cover, vegetation, and urban areas

## The geoai Super Resolution Function

The `geoai.super_resolution` module provides a `SuperResolutionModel` class that implements state-of-the-art super resolution algorithms:

- **ESRGAN**: Enhanced Super-Resolution Generative Adversarial Network - produces highly realistic results
- **SRCNN**: Super-Resolution Convolutional Neural Network - faster and more lightweight

### Key Parameters

- `model_type`: 'esrgan' or 'srcnn'
- `upscale_factor`: Scaling factor (2, 4, 8)
- `num_channels`: Number of input channels (3 for RGB, 4 for RGB+NIR)
- `device`: Computing device ('cuda', 'cpu', 'mps')

### Main Methods

- `enhance_image()`: Apply super resolution to an image
- `train()`: Train the model on custom data
- `load_model()`: Load pre-trained weights
- `evaluate()`: Calculate PSNR and SSIM metrics

## Install Package

To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
%pip install geoai-py

## Import Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import rasterio
from rasterio.transform import from_bounds
import torch

import geoai
from geoai.super_resolution import SuperResolutionModel, create_super_resolution_model

## Download Sample Data

We'll use a sample NAIP (National Agriculture Imagery Program) image for demonstration. This high-resolution aerial imagery will serve as our "ground truth" high-resolution image.

In [ ]:
# Download sample high-resolution NAIP imagery
naip_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_test.tif"
)
hr_image_path = geoai.download_file(naip_url)

print(f"Downloaded high-resolution image: {hr_image_path}")
geoai.get_raster_info(hr_image_path)

## Prepare Sample Data

For demonstration purposes, we'll create a low-resolution version of our high-resolution image by downsampling it. In real-world scenarios, you would start with actual low-resolution imagery.

In [ ]:
def create_low_res_version(hr_path, scale_factor=4, output_path=None):
    """
    Create a low-resolution version of a high-resolution image by downsampling.

    Args:
        hr_path: Path to high-resolution image
        scale_factor: Downsampling factor
        output_path: Path to save low-resolution image

    Returns:
        Path to low-resolution image
    """
    with rasterio.open(hr_path) as src:
        # Read RGB bands
        if src.count >= 3:
            hr_image = src.read([1, 2, 3])
        else:
            hr_image = src.read(1)
            hr_image = np.stack([hr_image, hr_image, hr_image])

        meta = src.meta.copy()
        transform = src.transform
        crs = src.crs

        # Downsample using bicubic interpolation
        from skimage.transform import resize

        lr_image = (
            resize(
                hr_image.transpose(1, 2, 0),
                (hr_image.shape[1] // scale_factor, hr_image.shape[2] // scale_factor),
                anti_aliasing=True,
                preserve_range=True,
            )
            .astype(np.uint8)
            .transpose(2, 0, 1)
        )

        # Update metadata for low-resolution image
        new_transform = from_bounds(
            transform.c,  # left
            transform.f - (transform.e * hr_image.shape[1]),  # bottom
            transform.c + (transform.a * hr_image.shape[2]),  # right
            transform.f,  # top
            lr_image.shape[2],
            lr_image.shape[1],
        )

        meta.update(
            {
                "height": lr_image.shape[1],
                "width": lr_image.shape[2],
                "transform": new_transform,
                "count": 3,
            }
        )

        if output_path is None:
            output_path = hr_path.replace(".tif", f"_lr_x{scale_factor}.tif")

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(lr_image)

        return output_path


# Create low-resolution version
lr_image_path = create_low_res_version(hr_image_path, scale_factor=4)
print(f"Created low-resolution image: {lr_image_path}")
geoai.get_raster_info(lr_image_path)

## Visualize Original vs Low-Resolution Images

Let's compare the high-resolution original with our artificially created low-resolution version.

In [ ]:
# Load and display images
def load_rgb_image(path):
    with rasterio.open(path) as src:
        if src.count >= 3:
            img = src.read([1, 2, 3])
        else:
            img = src.read(1)
            img = np.stack([img, img, img])
        return img.transpose(1, 2, 0)


hr_img = load_rgb_image(hr_image_path)
lr_img = load_rgb_image(lr_image_path)

# Display comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].imshow(hr_img)
axes[0].set_title(f"High-Resolution\n{hr_img.shape}")
axes[0].axis("off")

axes[1].imshow(lr_img)
axes[1].set_title(f"Low-Resolution\n{lr_img.shape}")
axes[1].axis("off")

plt.tight_layout()
plt.show()

print(
    f"Resolution improvement needed: {hr_img.shape[0] // lr_img.shape[0]}x in each dimension"
)
print(f"Total pixel increase: {(hr_img.shape[0] // lr_img.shape[0]) ** 2}x")

## Initialize Super Resolution Model

Now we'll create a super resolution model. For demonstration, we'll use the ESRGAN model with 4x upscaling. Note that without pre-trained weights, the model will produce results similar to bicubic interpolation, but the framework is ready for trained models.

In [ ]:
# Create ESRGAN model for 4x super resolution
sr_model = create_super_resolution_model(
    model_type="esrgan",
    upscale_factor=4,
    num_channels=3,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

print(f"Created {sr_model.model_type} model with {sr_model.upscale_factor}x upscaling")
print(f"Using device: {sr_model.device}")
print(f"Model parameters: {sum(p.numel() for p in sr_model.model.parameters()):,}")

## Apply Super Resolution

Now we'll apply the super resolution model to enhance our low-resolution image.

In [ ]:
# Apply super resolution
sr_output_path = lr_image_path.replace("_lr_x4.tif", "_sr_esrgan.tif")

print("Applying super resolution...")
sr_path = sr_model.enhance_image(
    input_path=lr_image_path,
    output_path=sr_output_path,
    tile_size=256,  # Process in tiles for large images
    overlap=32,
)

print(f"Super-resolved image saved to: {sr_path}")
geoai.get_raster_info(sr_path)

## Compare Results

Let's compare the original low-resolution image, the super-resolved output, and the ground truth high-resolution image.

In [ ]:
# Load super-resolved image
sr_img = load_rgb_image(sr_path)

# Display comparison
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Low-resolution (bicubic upsampled for comparison)
lr_upsampled = (
    np.array(plt.imread(lr_image_path.replace(".tif", "_lr_x4.tif")))
    if os.path.exists(lr_image_path.replace(".tif", "_lr_x4.tif"))
    else lr_img
)
if lr_upsampled.shape != hr_img.shape:
    from skimage.transform import resize

    lr_upsampled = resize(lr_img, hr_img.shape[:2], preserve_range=True).astype(
        np.uint8
    )

axes[0].imshow(lr_upsampled)
axes[0].set_title("Low-Res + Bicubic Upsampling")
axes[0].axis("off")

axes[1].imshow(sr_img)
axes[1].set_title("Super Resolution (ESRGAN)")
axes[1].axis("off")

axes[2].imshow(hr_img)
axes[2].set_title("Ground Truth (High-Res)")
axes[2].axis("off")

plt.tight_layout()
plt.show()

## Quantitative Evaluation

Let's calculate quantitative metrics to evaluate the super resolution performance. We'll compare PSNR (Peak Signal-to-Noise Ratio) and SSIM (Structural Similarity Index) between the different methods and the ground truth.

In [ ]:
def calculate_metrics(pred, target, data_range=255.0):
    """Calculate PSNR and SSIM metrics."""
    # Ensure same shape
    if pred.shape != target.shape:
        from skimage.transform import resize

        pred = resize(pred, target.shape, preserve_range=True)

    psnr = peak_signal_noise_ratio(target, pred, data_range=data_range)
    ssim = structural_similarity(target, pred, data_range=data_range, channel_axis=2)

    return psnr, ssim


# Calculate metrics
psnr_bicubic, ssim_bicubic = calculate_metrics(
    lr_upsampled.astype(float), hr_img.astype(float)
)
psnr_sr, ssim_sr = calculate_metrics(sr_img.astype(float), hr_img.astype(float))

print(f"Bicubic Upsampling - PSNR: {psnr_bicubic:.2f} dB, SSIM: {ssim_bicubic:.4f}")
print(f"Super Resolution   - PSNR: {psnr_sr:.2f} dB, SSIM: {ssim_sr:.4f}")
print(
    f"Improvement        - PSNR: {psnr_sr - psnr_bicubic:.2f} dB, SSIM: {ssim_sr - ssim_bicubic:.4f}"
)

# Create metrics visualization
methods = ["Bicubic", "Super Resolution"]
psnr_values = [psnr_bicubic, psnr_sr]
ssim_values = [ssim_bicubic, ssim_sr]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].bar(methods, psnr_values, color=["skyblue", "lightgreen"])
axes[0].set_title("PSNR Comparison")
axes[0].set_ylabel("PSNR (dB)")
axes[0].grid(True, alpha=0.3)

axes[1].bar(methods, ssim_values, color=["skyblue", "lightgreen"])
axes[1].set_title("SSIM Comparison")
axes[1].set_ylabel("SSIM")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Try Different Model Types

Let's compare the performance of different super resolution models.

In [ ]:
# Create SRCNN model for comparison
srcnn_model = create_super_resolution_model(
    model_type="srcnn",
    upscale_factor=4,
    num_channels=3,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Apply SRCNN
srcnn_output_path = lr_image_path.replace("_lr_x4.tif", "_sr_srcnn.tif")
srcnn_path = srcnn_model.enhance_image(
    input_path=lr_image_path, output_path=srcnn_output_path
)

# Load and evaluate
srcnn_img = load_rgb_image(srcnn_path)
psnr_srcnn, ssim_srcnn = calculate_metrics(
    srcnn_img.astype(float), hr_img.astype(float)
)

print(f"SRCNN Results - PSNR: {psnr_srcnn:.2f} dB, SSIM: {ssim_srcnn:.4f}")

# Compare all methods
methods = ["Bicubic", "SRCNN", "ESRGAN"]
psnr_all = [psnr_bicubic, psnr_srcnn, psnr_sr]
ssim_all = [ssim_bicubic, ssim_srcnn, ssim_sr]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].bar(methods, psnr_all, color=["skyblue", "orange", "lightgreen"])
axes[0].set_title("PSNR Comparison - All Methods")
axes[0].set_ylabel("PSNR (dB)")
axes[0].grid(True, alpha=0.3)

axes[1].bar(methods, ssim_all, color=["skyblue", "orange", "lightgreen"])
axes[1].set_title("SSIM Comparison - All Methods")
axes[1].set_ylabel("SSIM")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Error Handling and Edge Cases

Let's demonstrate how the super resolution function handles various edge cases and potential errors.

In [ ]:
# Test with invalid inputs
try:
    # Invalid model type
    invalid_model = create_super_resolution_model(model_type="invalid")
except ValueError as e:
    print(f"Expected error for invalid model type: {e}")

try:
    # Non-existent file
    sr_model.enhance_image("non_existent.tif")
except FileNotFoundError as e:
    print(f"Expected error for missing file: {e}")

# Test with different upscale factors
for factor in [2, 4, 8]:
    try:
        test_model = create_super_resolution_model(
            model_type="srcnn", upscale_factor=factor, num_channels=3
        )
        print(f"Successfully created model with {factor}x upscaling")
    except Exception as e:
        print(f"Error with {factor}x upscaling: {e}")

# Test memory constraints (small tile size)
large_sr_path = sr_model.enhance_image(
    input_path=lr_image_path,
    tile_size=128,  # Smaller tiles for memory efficiency
    overlap=16,
)
print(f"Processed with small tiles: {large_sr_path}")

## Training a Custom Model (Optional)

For demonstration purposes, we'll show how to train a super resolution model. Note that training requires significant computational resources and time, so this is commented out by default.

In [ ]:
# Create training data directory
train_dir = "sr_training_data"
os.makedirs(train_dir, exist_ok=True)

# Copy our high-res image to training directory
import shutil

shutil.copy(hr_image_path, os.path.join(train_dir, "sample_hr.tif"))

print(f"Training data prepared in: {train_dir}")

# Training code (commented out - requires significant compute)
"""
# Create training model
train_model = create_super_resolution_model(
    model_type='srcnn',  # Faster training than ESRGAN
    upscale_factor=4,
    num_channels=3
)

# Train the model
history = train_model.train(
    train_dir=train_dir,
    epochs=50,  # Increase for better results
    batch_size=8,
    learning_rate=1e-4,
    save_path='trained_sr_model.pth'
)

# Plot training history
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
if 'val_loss' in history:
    plt.plot(history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training History')
plt.grid(True, alpha=0.3)
plt.show()
"""

print("Training code is commented out. Uncomment to train a custom model.")
print("Training typically requires hours of GPU compute for good results.")

## Conclusion

This notebook demonstrated the comprehensive usage of the super resolution functionality in the `geoai` library. Here's a summary of what we covered:

### Key Takeaways

1. **Super Resolution in Geospatial AI**: SR can significantly enhance the resolution of satellite and aerial imagery, enabling better analysis of land cover, urban features, and environmental changes.

2. **Model Types**: 
   - **ESRGAN**: Better image quality, more computationally intensive
   - **SRCNN**: Faster inference, good for real-time applications

3. **Practical Usage**:
   - Easy model creation with `create_super_resolution_model()`
   - Flexible enhancement with `enhance_image()` method
   - Support for large images through tiled processing
   - Multiple upscale factors (2x, 4x, 8x)

4. **Evaluation Metrics**:
   - **PSNR**: Measures pixel-level accuracy
   - **SSIM**: Assesses structural similarity
   - Higher values indicate better performance

### Applications in Geospatial Analysis

- **Land Cover Classification**: Improved resolution leads to better classification accuracy
- **Urban Planning**: Enhanced detail for building and infrastructure analysis
- **Environmental Monitoring**: Better detection of small changes in vegetation and water bodies
- **Historical Data Enhancement**: Making older imagery comparable to modern high-resolution data

### Tips for Real-World Usage

1. **Model Selection**: Use ESRGAN for quality-critical applications, SRCNN for speed
2. **Training Data**: More diverse training data leads to better generalization
3. **Hardware**: GPU acceleration is recommended for large images
4. **Evaluation**: Always evaluate with domain-specific metrics, not just PSNR/SSIM
5. **Preprocessing**: Consider atmospheric correction and radiometric normalization
6. **Post-processing**: Apply super resolution before, not after, geometric corrections

### Limitations and Considerations

- **Training Requirements**: Models need extensive training for optimal performance
- **Computational Cost**: High-resolution outputs require significant compute resources
- **Artifacts**: Untrained models may introduce artifacts
- **Generalization**: Models trained on specific regions may not generalize well to others

For production use, consider training custom models on your specific data domain for best results.